# From Bellman Equation to Dynamic Programming

- 발표자: 석사과정 최선묵

## Markov Decision Process (MDP)

- $\mathcal{S}$: the state space
- $\mathcal{A}$: the action space
- $P$ : the transition probability
    - $P_{ss'}^a = P(S_{t+1}=s' | S_t =s, A_t = a)$
    - Assume Stationary Markov Process, that is, $P_{ss'}^a$ does not depend on time step $t$.
- $R$: the reward function
- $\gamma \in [0,1]$: discount factor

### The GOAL of Reinforcement Learning (Solving MDP)

- Given an MDP, we aim to find an optimal `policy` $\pi_\ast \colon \mathcal{S} \to \mathcal{A}$.
    - Policy decides which action the agent should choose for each state.
    - Policy can be either deterministic or stochastic.
        - Deterministic: $\pi(s) = a \in \mathcal{A}$ for each $s \in \mathcal{S}$
        - Stochastic: $\pi(a|s)$ a distribution
- `Optimal` in what sense?
    - An optimal policy maximizes the expected total reward.
- **Reward Hypothesis**
    - All goals can be described by the maximization of the expected value of the cumulative sum of rewards.

- Return $G_t$: the total discounted reward from time step $t$.
    - $G_t = R_{t+1} + \gamma R_{t+2} + \cdots = \sum_{k=0}^\infty \gamma^k R_{t+k+1}$, $\gamma \in [0,1]$
- Why discount?
    - mathematically convenient (converges if the reward function is bounded)
    - uncertainty of the future
    - immediate rewards may earn more interset than delayed rewards

### Value functions

- State-value function $v_\pi(s)$ for policy $\pi$
$$ v_\pi(s) = \mathbb{E}_\pi [G_t | S_t =s] $$

- Action-value function $q_\pi(s,a)$ for policy $\pi$
$$ q_\pi(s,a) = \mathbb{E}_\pi [G_t | S_t=s, A_t =a]$$

- By the definition, $v_\pi(s) = \sum_a \pi(a|s) q_\pi(s,a)$.

### Bellman Expectation Equation
- a recursive equation decomposing value function into immediate reward and discounted successor value.
\begin{align*}
v_\pi(s) &= \mathbb{E}_\pi[R_{t+1} + \gamma v_\pi(S_{t+1}) | S_t = s] \\
q_\pi(s,a) &= \mathbb{E}_\pi [R_{t+1} + \gamma q_\pi(S_{t+1}, A_{t+1}) | S_t = s, A_t=a)]
\end{align*}

### Optimal Value Functions and Policy
- Optimal state-value function $v_\ast(s) = \max_\pi v_\pi(s)$
- Optimal action-value function $q_\ast(s,a) = \max_\pi q_\pi(s,a)$
- Note that $\arg\max_\pi v_\pi(s)$ and $\arg\max_\pi v_\pi(s')$ may not the same.

#### Theorem
- Define a partial ordering $\pi' \geq \pi$ if $v_\pi'(s) \geq v_\pi(s)$ for all $s$.
    - There exists an optimal policy $\pi_\ast \geq \pi$ for all $\pi$.
    - All optimal policies achieve the optimal state-value function $v_{\pi_\ast}(s) = v_\ast(s)$.
    - All optimal policies achieve the optimal action-value function $q_{\pi_\ast}(s,a) = q_\ast(s,a)$.
- The theorem says that $$\pi_\ast = \arg\max_\pi v_\pi(s), \quad \forall s \in \mathcal{S}.$$

#### Optimal Policy
- An optimal policy can be found by maximizing over $q_\ast(s,a)$.
$$ \pi_\ast(a|s) = \begin{cases} 1 & \text{if} \;\; a = \arg\max_a q_\ast(s,a) \\ 0 & \text{otherwise} \end{cases} \quad \text{or} \quad \pi_\ast(s) = \arg\max_a q_\ast(s,a)$$
- $v_\ast(s) = \max_a q_\ast(s,a)$
- $v_\ast(s)$ can be obtained directly from $q_\ast(s,a)$
- $q_\ast(s,a)$ cannot be obtained directly from $v_\ast(s)$.

### Bellman Optimality Equation 

\begin{align*}
v_\ast(s) &= \max_a \sum_{s',r} p(s',r | s,a)[r+\gamma v_\ast(s')] \\
q_\ast(s,a) &= \sum_{s',r} p(s',r | s,a)[r+\gamma \max_{a'}q_\ast(s',a')] 
\end{align*}

## Value Iteration

We want to find an `optimal value function` using iterative algorithm.
- Bellman optimal equation
    $$V^\ast(s) = \max_a \sum_{s',r} p(s',r | s,a)[r+\gamma V^\ast(s')]$$

`Update Equation` : $V_{k+1}(s) \leftarrow \max_a \sum_{s',r} p(s',r | s,a)[r+\gamma V_k(s')]$

1. Initialize $V_0(s) = 0$ for all states $s$.
2. Update $V_{k+1}(s)$ iteratively from all $V_k(s')$ (full backup) until convergence.
    - synchronous backups: compute $V_{k+1}(s)$ for all $s$ and update simultaneously
    - asynchronous backups: compute $V_{k+1}(s)$ for one $s$ and update it immediately
3. Compute the optimal policy $\pi_\ast$
$$ \pi_\ast(s) = \arg\max_a \sum_{s',r} p(s',r | s,a)[r+\gamma V^\ast(s')]$$

### Example 1. Driving Car

![image](./driving_car.png)

In [1]:
import numpy as np

In [2]:
## Environment 
# 3 states: cool(0), warm(1), overheated(2) 
# 2 actions: slow(0), fast(1)
# Going faster gets double reward with one exception.

# Transition Probabilities and Rewards for each action
# P[action][state, next_state] = [probability, reward]

P = {}
P[0] = np.array([[[1,1], [0,1], [0,0]],
                [[0.5,1], [0.5,1], [0,0]],
                [[0,0], [0,0], [1,0]]])

P[1] = np.array([[[0.5,2], [0.5,2], [0,0]],
                [[0,2], [0,2], [1,-10]],
                [[0,0], [0,0], [1,0]]])

- `Update Equation` : $V_{k+1}(s) \leftarrow \max_a \sum_{s',r} p(s',r | s,a)[r+\gamma V_k(s')]$
- `Policy` : $ \pi_\ast(s) = \arg\max_a \sum_{s',r} p(s',r | s,a)[r+\gamma V^\ast(s')]$

In [3]:
def next_value_ftn_vi(state, values, gamma):         # states: 0, 1, 2
    temp_slow = 0
    temp_fast = 0
    for next_state in range(3):
        temp_slow += P[0][state, next_state][0] * (P[0][state, next_state][1] + gamma * values[next_state])
        temp_fast += P[1][state, next_state][0] * (P[1][state, next_state][1] + gamma * values[next_state])
    
    values = [temp_slow, temp_fast]
    return max(values), np.argmax(values)

In [4]:
# Define Value function with a dictionary. 
V = {}

# Initialize the value function for each state (except terminal state) randomly.
for i in range(2):
    V[i] = np.random.random()
V[2] = 0

# Set hyperparameters
threshold = 0.001
gamma = 0.9

# Policy
Pi = {}
for i in range(3):
    Pi[i] = int(np.random.randint(low=0, high=2, size=1))

stage = 0
while True:
    delta = 0
    print('stage: %d, V[0]= %.3f, V[1]= %.3f, V[2]=%.3f, Policy: (%d, %d, %d)' %(stage, V[0], V[1], V[2], Pi[0], Pi[1], Pi[2]))
    temp = []
    for i in range(3):
        temp.append(V[i])
    for j in range(3):
        v = V[j]
        V[j], Pi[j] = next_value_ftn_vi(j, temp, gamma)
        delta = max(delta, abs(v - V[j]))
        
    if delta < threshold:
        break
    else:
        stage += 1
        continue

stage: 0, V[0]= 0.057, V[1]= 0.734, V[2]=0.000, Policy: (1, 0, 1)
stage: 1, V[0]= 2.356, V[1]= 1.356, V[2]=0.000, Policy: (1, 0, 0)
stage: 2, V[0]= 3.670, V[1]= 2.670, V[2]=0.000, Policy: (1, 0, 0)
stage: 3, V[0]= 4.853, V[1]= 3.853, V[2]=0.000, Policy: (1, 0, 0)
stage: 4, V[0]= 5.918, V[1]= 4.918, V[2]=0.000, Policy: (1, 0, 0)
stage: 5, V[0]= 6.876, V[1]= 5.876, V[2]=0.000, Policy: (1, 0, 0)
stage: 6, V[0]= 7.738, V[1]= 6.738, V[2]=0.000, Policy: (1, 0, 0)
stage: 7, V[0]= 8.515, V[1]= 7.515, V[2]=0.000, Policy: (1, 0, 0)
stage: 8, V[0]= 9.213, V[1]= 8.213, V[2]=0.000, Policy: (1, 0, 0)
stage: 9, V[0]= 9.842, V[1]= 8.842, V[2]=0.000, Policy: (1, 0, 0)
stage: 10, V[0]= 10.408, V[1]= 9.408, V[2]=0.000, Policy: (1, 0, 0)
stage: 11, V[0]= 10.917, V[1]= 9.917, V[2]=0.000, Policy: (1, 0, 0)
stage: 12, V[0]= 11.375, V[1]= 10.375, V[2]=0.000, Policy: (1, 0, 0)
stage: 13, V[0]= 11.788, V[1]= 10.788, V[2]=0.000, Policy: (1, 0, 0)
stage: 14, V[0]= 12.159, V[1]= 11.159, V[2]=0.000, Policy: (1, 0, 

### Example 2. Grid World

- States (Total 11 states)
    - 0 : (1,1)
    - 1 : (2,1)
    - 2 : (3,1)
    - 3 : (4,1)
    - 4 : (1,2)
    - 5 : (3,2)
    - 6 : (4,2) -> **Terminal state**
    - 7 : (1,3)
    - 8 : (2,3)
    - 9 : (3,3) 
    - 10 : (4,3) -> **Terminal state**
- Actions (Total 4 actions)
    - 0 : North
    - 1 : South
    - 2 : East
    - 3 : West
- Noisy environment, e.g., $P_{(3,1)(3,2)}^{North} = 0.8$, $P_{(3,1)(2,1)}^{North} = 0.1$, $P_{(3,1)(4,1)}^{North} = 0.1$
- Reward
    - When an agent reaches the state (4,2), then it gains $-1$ reward.
    - When an agent reaches the state (4,3), then it gains $+1$ reward.
    - An agent gains small negative reward $c$ for each step, except for the case when the agent reaches the terminal state.


![image](./grid_world.png)

In [5]:
# Transition Probabilities and Rewards for each action
# P[action][state, next_state] = [probability, reward]

state_dim = 11
action_dim = 4
c=-0.4

Action = {}
Action[0] = 'North'
Action[1] = 'South'
Action[2] = 'East'
Action[3] = 'West'

P = {}
for i in range(action_dim):
    P[i] = np.zeros((state_dim,state_dim,2))

# North
P[0][0,0] = [0.1, c]
P[0][0,1] = [0.1, c]
P[0][0,4] = [0.8, c]
P[0][1,0] = [0.1, c]
P[0][1,1] = [0.8, c]
P[0][1,2] = [0.1, c]
P[0][2,1] = [0.1, c]
P[0][2,5] = [0.8, c]
P[0][2,3] = [0.1, c]
P[0][3,2] = [0.1, c]
P[0][3,3] = [0.1, c]
P[0][3,6] = [0.8, -1]
P[0][4,4] = [0.2, c]
P[0][4,7] = [0.8, c]
P[0][5,5] = [0.1, c]
P[0][5,9] = [0.8, c]
P[0][5,6] = [0.1, -1]
P[0][6,6] = [1, 0]
P[0][7,7] = [0.9, c]
P[0][7,8] = [0.1, c]
P[0][8,7] = [0.1, c]
P[0][8,8] = [0.8, c]
P[0][8,9] = [0.1, c]
P[0][9,8] = [0.1, c]
P[0][9,9] = [0.8, c]
P[0][9,10] = [0.1, 1]
P[0][10,10] = [1,0]

# South
P[1][0,0] = [0.9, c]
P[1][0,1] = [0.1, c]
P[1][1,0] = [0.1, c]
P[1][1,1] = [0.8, c]
P[1][1,2] = [0.1, c]
P[1][2,1] = [0.1, c]
P[1][2,2] = [0.8, c]
P[1][2,3] = [0.1, c]
P[1][3,2] = [0.1, c]
P[1][3,3] = [0.9, c]
P[1][4,0] = [0.8, c]
P[1][4,4] = [0.2, c]
P[1][5,2] = [0.8, c]
P[1][5,5] = [0.1, c]
P[1][5,6] = [0.1, -1]
P[1][6,6] = [1, 0]
P[1][7,4] = [0.8, c]
P[1][7,7] = [0.1, c]
P[1][7,8] = [0.1, c]
P[1][8,7] = [0.1, c]
P[1][8,8] = [0.8, c]
P[1][8,9] = [0.1, c]
P[1][9,8] = [0.1, c]
P[1][9,9] = [0.8, c]
P[1][9,10] = [0.1, 1]
P[1][10,10] = [1, 0]

# East
P[2][0,0] = [0.1, c]
P[2][0,1] = [0.8, c]
P[2][0,4] = [0.1, c]
P[2][1,1] = [0.2, c]
P[2][1,2] = [0.8, c]
P[2][2,2] = [0.1, c]
P[2][2,3] = [0.8, c]
P[2][2,5] = [0.1, c]
P[2][3,3] = [0.9, c]
P[2][3,6] = [0.1, -1]
P[2][4,0] = [0.1, c]
P[2][4,4] = [0.8, c]
P[2][4,7] = [0.1, c]
P[2][5,2] = [0.1, c]
P[2][5,6] = [0.8, -1]
P[2][5,9] = [0.1, c]
P[2][6,6] = [1, 0]
P[2][7,4] = [0.1, c]
P[2][7,7] = [0.1, c]
P[2][7,8] = [0.8, c]
P[2][8,8] = [0.2, c]
P[2][8,9] = [0.8, c]
P[2][9,5] = [0.1, c]
P[2][9,9] = [0.1, c]
P[2][9,10] = [0.8, 1]
P[2][10,10] = [1, 0]

# West
P[3][0,0] = [0.9, c]
P[3][0,4] = [0.1, c]
P[3][1,0] = [0.8, c]
P[3][1,1] = [0.2, c]
P[3][2,1] = [0.8, c]
P[3][2,2] = [0.1, c]
P[3][2,5] = [0.1, c]
P[3][3,2] = [0.8, c]
P[3][3,3] = [0.1, c]
P[3][3,6] = [0.1, -1]
P[3][4,0] = [0.1, c]
P[3][4,4] = [0.8, c]
P[3][4,7] = [0.1, c]
P[3][5,2] = [0.1, c]
P[3][5,5] = [0.8, c]
P[3][5,9] = [0.1, c]
P[3][6,6] = [1, 0]
P[3][7,4] = [0.1, c]
P[3][7,7] = [0.9, c]
P[3][8,7] = [0.8, c]
P[3][8,8] = [0.2, c]
P[3][9,5] = [0.1, c]
P[3][9,8] = [0.8, c]
P[3][9,9] = [0.1, c]
P[3][10,10] = [1, 0]

In [6]:
def next_value_ftn_vi_2(state, values, gamma, state_dim, action_dim):         # states: 0 ~ 10, actions: 0 ~ 3
    temp = {}
    for a in range(action_dim):
        temp[a] = 0
        for next_state in range(state_dim):
            temp[a] += P[a][state, next_state][0] * (P[a][state, next_state][1] + gamma * values[next_state])
            
    # return maximum value and the corresponding action
    return max(temp.values()), max(temp, key=temp.get)

In [7]:
# Define Value function with a dictionary. 
V = {}

# Initialize the value function for each state.
for i in range(state_dim):
    V[i] = np.random.random()
V[6] = 0
V[10] = 0

# Set hyperparameters
threshold = 0.0001
gamma = 1

# Policy
Pi = {}
for i in range(state_dim):
    Pi[i] = int(np.random.randint(low=0, high=action_dim, size=1))

stage = 0
while True:
    delta = 0
    temp = []
    for i in range(state_dim):
        temp.append(V[i])
    for j in range(state_dim):
        v = V[j]
        V[j], Pi[j] = next_value_ftn_vi_2(j, temp, gamma, state_dim, action_dim)
        delta = max(delta, abs(v - V[j]))
        
    if delta < threshold:
        for i in range(state_dim):
            print('V[%d] = %.4f, best action: %s' %(i, V[i], Action[Pi[i]]))
        break
    else:
        stage += 1
        print('Stage %d'%stage)
        continue

Stage 1
Stage 2
Stage 3
Stage 4
Stage 5
Stage 6
Stage 7
Stage 8
Stage 9
Stage 10
Stage 11
Stage 12
Stage 13
Stage 14
V[0] = -1.2002, best action: North
V[1] = -0.8989, best action: East
V[2] = -0.3989, best action: North
V[3] = -0.8657, best action: West
V[4] = -0.7379, best action: North
V[5] = 0.2219, best action: North
V[6] = 0.0000, best action: North
V[7] = -0.2378, best action: East
V[8] = 0.3247, best action: East
V[9] = 0.8247, best action: East
V[10] = 0.0000, best action: North


## Policy Iteration

Policy Iteration repeats `policy evaluation` and `policy improvement` until convergence

`Policy Evaluation`: computing $V^\pi$ from the deterministic policy $\pi$ using **Bellman Expectation Equation**
- `Update Equation`: $V_{k+1}(s) \leftarrow \sum_{s',r} p(s',r |s,\pi(s))[r+\gamma V_k(s')]$
1. Initialize $V_0(s)=0$ for all states $s$.
2. Update every $V_{k+1}(s)$ from all $V_k(s')$ until convergence to $V^\pi$.

`Policy Improvement`: improving $\pi$ to $\pi'$ by greedy policy based on $V^\pi$.
- `Update Equation`: $\pi'(s) = \arg\max_a \sum_{s',r} p(s',r|s,a)[r+\gamma V^\pi(s')] = \arg\max_a Q^\pi(s,a)$

Notice that $Q^\pi(s,\pi'(s)) \geq V^\pi(s) = \sum_a \pi(a|s) Q^\pi(s,a)$.

### Policy Improvement Theorem

Let $\pi$ and $\pi'$ be two policies. If $Q^\pi(s,\pi'(s)) \geq V^\pi(s)$ for all $s \in \mathcal{S}$, then $V^{\pi'}(s) \geq V^\pi(s)$ for all $s \in \mathcal{S}$. This implies that $\pi'$ is a better policy than $\pi$.

### Example 1. Driving Car

In [8]:
import numpy as np

In [9]:
## Environment 
# 3 states: cool(0), warm(1), overheated(2) 
# 2 actions: slow(0), fast(1)
# Going faster gets double reward with one exception.

# Transition Probabilities and Rewards for each action
# P[action][state, next_state] = [probability, reward]

P = {}
P[0] = np.array([[[1,1], [0,1], [0,0]],
                [[0.5,1], [0.5,1], [0,0]],
                [[0,0], [0,0], [1,0]]])

P[1] = np.array([[[0.5,2], [0.5,2], [0,0]],
                [[0,2], [0,2], [1,-10]],
                [[0,0], [0,0], [1,0]]])

- `Update value`: $V_{k+1}(s) \leftarrow \sum_{s',r} p(s',r |s,\pi(s))[r+\gamma V_k(s')]$
- `Update policy`: $\pi'(s) = \arg\max_a \sum_{s',r} p(s',r|s,a)[r+\gamma V^\pi(s')] = \arg\max_a Q^\pi(s,a)$

In [10]:
def next_value_ftn_pi(state, values, action, gamma):         # states: 0, 1, 2   actions: 0, 1
    temp = 0
    for i in range(3):
        temp += P[action][state, i][0] * (P[action][state, i][1] + gamma * values[i])
    return temp

def find_action(state, values, gamma):         # states: 0, 1, 2
    temp_slow = 0
    temp_fast = 0
    for next_state in range(3):
        temp_slow += P[0][state, next_state][0] * (P[0][state, next_state][1] + gamma * values[next_state])
        temp_fast += P[1][state, next_state][0] * (P[1][state, next_state][1] + gamma * values[next_state])
    
    values = [temp_slow, temp_fast]
    return np.argmax(values)

In [11]:
# Define Value function with a dictionary. 
V = {}

# Initialize the value function for each state.
for i in range(2):
    V[i] = np.random.random()
V[2] = 0

# Set hyperparameters
threshold = 0.01
gamma = 0.9

# Policy
Pi = {}
for i in range(3):
    Pi[i] = int(np.random.randint(low=0, high=2, size=1))

stage = 0
while True:
    print('stage: %d, V[0]= %.3f, V[1]= %.3f, V[2]=%.3f, Policy: (%d, %d, %d)' %(stage, V[0], V[1], V[2], Pi[0], Pi[1], Pi[2]))

    ########## Policy Evaluation ############
    print('Evaluation Phase')
    counter = 0
    while True:
        delta = 0
        temp = []
        for i in range(3):
            temp.append(V[i])
        for j in range(3):
            v = V[j]
            V[j] = next_value_ftn_pi(j, temp, Pi[j], gamma)
            delta = max(delta, abs(v - V[j]))
            
        if delta < threshold:
            break
        else:
            counter += 1
            print('counter: %d, V[0]= %.3f, V[1]= %.3f, V[2]=%.3f' %(counter, V[0], V[1], V[2]))
            continue
    #########################################

    ########## Policy Improvement ###########
    print('Improvement Phase')
    policy_stable = True
    for i in range(3):
        old_action = Pi[i]
        Pi[i] = find_action(i, V, gamma)
        if old_action != Pi[i]:
            policy_stable = False
    
    if policy_stable:
        print('Final Values and an optimal policy. V[0]= %.4f, V[1]=%.4f, V[2]=%.4f, optimal policy: (%d, %d, %d)' %(V[0], V[1], V[2], Pi[0], Pi[1], Pi[2]))
        break
    else:
        stage += 1
        print('Go back to Evaluation Phase.')
        print('#'*50)
        continue
    #######################################

stage: 0, V[0]= 0.893, V[1]= 0.750, V[2]=0.000, Policy: (0, 1, 1)
Evaluation Phase
counter: 1, V[0]= 1.803, V[1]= -10.000, V[2]=0.000
counter: 2, V[0]= 2.623, V[1]= -10.000, V[2]=0.000
counter: 3, V[0]= 3.361, V[1]= -10.000, V[2]=0.000
counter: 4, V[0]= 4.025, V[1]= -10.000, V[2]=0.000
counter: 5, V[0]= 4.622, V[1]= -10.000, V[2]=0.000
counter: 6, V[0]= 5.160, V[1]= -10.000, V[2]=0.000
counter: 7, V[0]= 5.644, V[1]= -10.000, V[2]=0.000
counter: 8, V[0]= 6.080, V[1]= -10.000, V[2]=0.000
counter: 9, V[0]= 6.472, V[1]= -10.000, V[2]=0.000
counter: 10, V[0]= 6.824, V[1]= -10.000, V[2]=0.000
counter: 11, V[0]= 7.142, V[1]= -10.000, V[2]=0.000
counter: 12, V[0]= 7.428, V[1]= -10.000, V[2]=0.000
counter: 13, V[0]= 7.685, V[1]= -10.000, V[2]=0.000
counter: 14, V[0]= 7.917, V[1]= -10.000, V[2]=0.000
counter: 15, V[0]= 8.125, V[1]= -10.000, V[2]=0.000
counter: 16, V[0]= 8.312, V[1]= -10.000, V[2]=0.000
counter: 17, V[0]= 8.481, V[1]= -10.000, V[2]=0.000
counter: 18, V[0]= 8.633, V[1]= -10.000, V

### Example 2. Grid World

In [12]:
# Transition Probabilities and Rewards for each action
# P[action][state, next_state] = [probability, reward]

state_dim = 11
action_dim = 4
c=-0.4

Action = {}
Action[0] = 'North'
Action[1] = 'South'
Action[2] = 'East'
Action[3] = 'West'

P = {}
for i in range(action_dim):
    P[i] = np.zeros((state_dim,state_dim,2))

# North
P[0][0,0] = [0.1, c]
P[0][0,1] = [0.1, c]
P[0][0,4] = [0.8, c]
P[0][1,0] = [0.1, c]
P[0][1,1] = [0.8, c]
P[0][1,2] = [0.1, c]
P[0][2,1] = [0.1, c]
P[0][2,5] = [0.8, c]
P[0][2,3] = [0.1, c]
P[0][3,2] = [0.1, c]
P[0][3,3] = [0.1, c]
P[0][3,6] = [0.8, -1]
P[0][4,4] = [0.2, c]
P[0][4,7] = [0.8, c]
P[0][5,5] = [0.1, c]
P[0][5,9] = [0.8, c]
P[0][5,6] = [0.1, -1]
P[0][6,6] = [1, 0]
P[0][7,7] = [0.9, c]
P[0][7,8] = [0.1, c]
P[0][8,7] = [0.1, c]
P[0][8,8] = [0.8, c]
P[0][8,9] = [0.1, c]
P[0][9,8] = [0.1, c]
P[0][9,9] = [0.8, c]
P[0][9,10] = [0.1, 1]
P[0][10,10] = [1,0]

# South
P[1][0,0] = [0.9, c]
P[1][0,1] = [0.1, c]
P[1][1,0] = [0.1, c]
P[1][1,1] = [0.8, c]
P[1][1,2] = [0.1, c]
P[1][2,1] = [0.1, c]
P[1][2,2] = [0.8, c]
P[1][2,3] = [0.1, c]
P[1][3,2] = [0.1, c]
P[1][3,3] = [0.9, c]
P[1][4,0] = [0.8, c]
P[1][4,4] = [0.2, c]
P[1][5,2] = [0.8, c]
P[1][5,5] = [0.1, c]
P[1][5,6] = [0.1, -1]
P[1][6,6] = [1, 0]
P[1][7,4] = [0.8, c]
P[1][7,7] = [0.1, c]
P[1][7,8] = [0.1, c]
P[1][8,7] = [0.1, c]
P[1][8,8] = [0.8, c]
P[1][8,9] = [0.1, c]
P[1][9,8] = [0.1, c]
P[1][9,9] = [0.8, c]
P[1][9,10] = [0.1, 1]
P[1][10,10] = [1, 0]

# East
P[2][0,0] = [0.1, c]
P[2][0,1] = [0.8, c]
P[2][0,4] = [0.1, c]
P[2][1,1] = [0.2, c]
P[2][1,2] = [0.8, c]
P[2][2,2] = [0.1, c]
P[2][2,3] = [0.8, c]
P[2][2,5] = [0.1, c]
P[2][3,3] = [0.9, c]
P[2][3,6] = [0.1, -1]
P[2][4,0] = [0.1, c]
P[2][4,4] = [0.8, c]
P[2][4,7] = [0.1, c]
P[2][5,2] = [0.1, c]
P[2][5,6] = [0.8, -1]
P[2][5,9] = [0.1, c]
P[2][6,6] = [1, 0]
P[2][7,4] = [0.1, c]
P[2][7,7] = [0.1, c]
P[2][7,8] = [0.8, c]
P[2][8,8] = [0.2, c]
P[2][8,9] = [0.8, c]
P[2][9,5] = [0.1, c]
P[2][9,9] = [0.1, c]
P[2][9,10] = [0.8, 1]
P[2][10,10] = [1, 0]

# West
P[3][0,0] = [0.9, c]
P[3][0,4] = [0.1, c]
P[3][1,0] = [0.8, c]
P[3][1,1] = [0.2, c]
P[3][2,1] = [0.8, c]
P[3][2,2] = [0.1, c]
P[3][2,5] = [0.1, c]
P[3][3,2] = [0.8, c]
P[3][3,3] = [0.1, c]
P[3][3,6] = [0.1, -1]
P[3][4,0] = [0.1, c]
P[3][4,4] = [0.8, c]
P[3][4,7] = [0.1, c]
P[3][5,2] = [0.1, c]
P[3][5,5] = [0.8, c]
P[3][5,9] = [0.1, c]
P[3][6,6] = [1, 0]
P[3][7,4] = [0.1, c]
P[3][7,7] = [0.9, c]
P[3][8,7] = [0.8, c]
P[3][8,8] = [0.2, c]
P[3][9,5] = [0.1, c]
P[3][9,8] = [0.8, c]
P[3][9,9] = [0.1, c]
P[3][10,10] = [1, 0]

In [13]:
def next_value_ftn_pi_2(state, values, action, gamma, state_dim):         # states: 0~10   actions: 0~3
    temp = 0
    for next_state in range(state_dim):
        temp += P[action][state, next_state][0] * (P[action][state, next_state][1] + gamma * values[next_state])
    return temp

def find_action_2(state, values, gamma, state_dim, action_dim):         # states: 0~10
    temp = {}
    for a in range(action_dim):
        temp[a] = 0
        for next_state in range(state_dim):
            temp[a] += P[a][state, next_state][0] * (P[a][state, next_state][1] + gamma * values[next_state])
            
    # return the action that maximizes the value
    return max(temp, key=temp.get)

In [16]:
# Define Value function with a dictionary. 
V = {}

# Initialize the value function for each state.
for i in range(state_dim):
    V[i] = np.random.random()
V[6] = 0
V[10] = 0

# Set hyperparameters
threshold = 0.001
gamma = 1

# Policy
Pi = {}
for i in range(state_dim):
    Pi[i] = int(np.random.randint(low=0, high=action_dim, size=1))

stage = 1
while True:
    ########## Policy Evaluation ############
    print('Evaluation Phase Stage %d' %stage)
    print('Current policy:', (Pi[0], Pi[1], Pi[2], Pi[3], Pi[4], Pi[5], Pi[6], Pi[7], Pi[8], Pi[9], Pi[10]))
    counter = 0
    while True:
        delta = 0
        temp = []
        for i in range(state_dim):
            temp.append(V[i])
        for j in range(state_dim):
            v = V[j]
            V[j] = next_value_ftn_pi_2(j, temp, Pi[j], gamma, state_dim)
            delta = max(delta, abs(v - V[j]))
            
        if delta < threshold:
            break
        else:
            counter += 1
            print('Evaluation counter %d' %counter)
            continue
    #########################################

    ########## Policy Improvement ###########
    print('Improvement Phase')
    policy_stable = True
    for i in range(state_dim):
        old_action = Pi[i]
        Pi[i] = find_action_2(i, V, gamma, state_dim, action_dim)
        if old_action != Pi[i]:
            policy_stable = False
    
    if policy_stable:
        for i in range(state_dim):
            print('V[%d] = %.4f, best action: %s' %(i, V[i], Action[Pi[i]]))
        break
    else:
        stage += 1
        print('Go back to Evaluation Phase.')
        print('#'*50)
        continue
    #######################################

Evaluation Phase Stage 1
Current policy: (1, 0, 1, 0, 3, 1, 3, 2, 2, 2, 0)
Evaluation counter 1
Evaluation counter 2
Evaluation counter 3
Evaluation counter 4
Evaluation counter 5
Evaluation counter 6
Evaluation counter 7
Evaluation counter 8
Evaluation counter 9
Evaluation counter 10
Evaluation counter 11
Evaluation counter 12
Evaluation counter 13
Evaluation counter 14
Evaluation counter 15
Evaluation counter 16
Evaluation counter 17
Evaluation counter 18
Evaluation counter 19
Evaluation counter 20
Evaluation counter 21
Evaluation counter 22
Evaluation counter 23
Evaluation counter 24
Evaluation counter 25
Evaluation counter 26
Evaluation counter 27
Evaluation counter 28
Evaluation counter 29
Evaluation counter 30
Evaluation counter 31
Evaluation counter 32
Evaluation counter 33
Evaluation counter 34
Evaluation counter 35
Evaluation counter 36
Evaluation counter 37
Evaluation counter 38
Evaluation counter 39
Evaluation counter 40
Evaluation counter 41
Evaluation counter 42
Evaluation

In [17]:
for i in range(state_dim):
    print('V[%d] = %.4f, best action: %s' %(i, V[i], Action[Pi[i]]))

V[0] = -1.2004, best action: North
V[1] = -0.8997, best action: East
V[2] = -0.3992, best action: North
V[3] = -0.8663, best action: West
V[4] = -0.7378, best action: North
V[5] = 0.2219, best action: North
V[6] = 0.0000, best action: North
V[7] = -0.2378, best action: East
V[8] = 0.3247, best action: East
V[9] = 0.8247, best action: East
V[10] = 0.0000, best action: North
